In [1]:
from __future__ import print_function
import os

import numpy as np
import zipfile
import tarfile
from six.moves.urllib.request import urlretrieve
import shutil 
import random

import string
import tensorflow as tf

# Dirs - must be absolute paths!
LOG_DIR = '/tmp/tf/ptb_char_lstm_mann_lrua_shared_memory/h64b10s1lm10/'
# Local dir where PTB files will be stored.
PTB_DIR = '/home/tkornuta/data/ptb/'

# Filenames.
TRAIN = "ptb.train.txt"
VALID = "ptb.valid.txt"
TEST = "ptb.test.txt"

# Size of the hidden state
HIDDEN_SIZE = 64

# Size of batch
BATCH_SIZE = 10

# Length of sequence (=  number of units of controller (recurrent layer))
SEQ_LENGTH = 1

#### MANN-related parameters.
# Size of the local memory of each cell.
MEMORY_SLOTS = 10

# Number of smallest elements - used in LRUA scheme.
N_SMALLEST = 2

# "Update weight decay".
GAMMA = 0.1


### Check/maybe download PTB.

In [2]:
def maybe_download_ptb(path, 
                       filename='simple-examples.tgz', 
                       url='http://www.fit.vutbr.cz/~imikolov/rnnlm/', 
                       expected_bytes =34869662):
  # Eventually create the PTB dir.
  if not tf.gfile.Exists(path):
    tf.gfile.MakeDirs(path)
  """Download a file if not present, and make sure it's the right size."""
  _filename = path+filename
  if not os.path.exists(_filename):
    print('Downloading %s...' % filename)
    _filename, _ = urlretrieve(url+filename, _filename)
  statinfo = os.stat(_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', (_filename), '(', statinfo.st_size, ')')
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + _filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download_ptb(PTB_DIR)

Found and verified /home/tkornuta/data/ptb/simple-examples.tgz ( 34869662 )


### Extract dataset-related files from the PTB archive.

In [3]:
def extract_ptb(path, filename='simple-examples.tgz', files=["ptb.train.txt", "ptb.valid.txt", "ptb.test.txt", 
                                       "ptb.char.train.txt", "ptb.char.valid.txt", "ptb.char.test.txt"]):
    """Extracts files from PTB archive."""
    # Extract
    tar = tarfile.open(path+filename)
    tar.extractall(path)
    tar.close()
    # Copy files
    for file in files:
        shutil.copyfile(PTB_DIR+"simple-examples/data/"+file, PTB_DIR+file)
    # Delete directory
    shutil.rmtree(PTB_DIR+"simple-examples/")        

extract_ptb(PTB_DIR)

### Load train, valid and test texts.

In [4]:
def read_data(filename, path):
    with open(path+filename, 'r') as myfile:
        data=myfile.read()# .replace('\n', '')
        return data

train_text = read_data(TRAIN, PTB_DIR)
train_size=len(train_text)
print(train_size, train_text[:100])

valid_text = read_data(VALID, PTB_DIR)
valid_size=len(valid_text)
print(valid_size, valid_text[:64])

test_text = read_data(TEST, PTB_DIR)
test_size=len(test_text)
print(test_size, test_text[:64])

5101618  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote
399782  consumers may want to move their telephones a little closer to 
449945  no it was n't black monday 
 but while the new york stock excha


### Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = 59 # [A-Z] + [a-z] + ' ' +few 'in between; + punctuation
first_letter = ord(string.ascii_uppercase[0]) # ascii_uppercase before lowercase! 
print("vocabulary size = ", vocabulary_size)
print(first_letter)

def char2id(char):
  """ Converts char to id (int) with one-hot encoding handling of unexpected characters"""
  if char in string.ascii_letters:# or char in string.punctuation or char in string.digits:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    # print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  """ Converts single id (int) to character"""
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

#print(len(string.punctuation))
#for i in string.ascii_letters:
#    print (i, char2id(i))


print(char2id('a'), char2id('A'), char2id('z'), char2id('Z'), char2id(' '), char2id('ï'))
print(id2char(char2id('a')), id2char(char2id('A')))
#print(id2char(65), id2char(33), id2char(90), id2char(58), id2char(0))
#bankno
sample = np.zeros(shape=(1, vocabulary_size), dtype=np.float)
sample[0, char2id(' ')] = 1.0
print(sample)

vocabulary size =  59
65
33 1 58 26 0 0
a A
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]


### Helper class for batch generation

In [6]:
class BatchGenerator(object):
  def __init__(self, text, batch_size, seq_length, vocab_size):
    """
    Initializes the batch generator object. Stores the variables and first "letter batch".
    text is text to be processed
    batch_size is size of batch (number of samples)
    seq_length represents the length of sequence
    vocab_size is number of words in vocabulary (assumes one-hot encoding)
    """
    # Store input parameters.
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._seq_length = seq_length
    self._vocab_size = vocab_size
    # Divide text into segments depending on number of batches, each segment determines a cursor position for a batch.
    segment = self._text_size // batch_size
    # Set initial cursor position.
    self._cursor = [ offset * segment for offset in range(batch_size)]
    # Store first "letter batch".
    self._last_letter_batch = self._next_letter_batch()
  
  def _next_letter_batch(self):
    """
    Returns a batch containing of encoded single letters depending on the current batch 
    cursor positions in the data.
    Returned "letter batch" is of size batch_size x vocab_size
    """
    letter_batch = np.zeros(shape=(self._batch_size, self._vocab_size), dtype=np.float)
    # Iterate through "samples"
    for b in range(self._batch_size):
      # Set 1 in position pointed out by one-hot char encoding.
      letter_batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return letter_batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    # First add last letter from previous batch (the "additional one").
    batches = [self._last_letter_batch]
    for step in range(self._seq_length):
      batches.append(self._next_letter_batch())
    # Store last "letter batch" for next batch.
    self._last_letter_batch = batches[-1]
    return batches


In [25]:
# Trick - override first 10 chars
#list1 = list(train_text)
#for i in range(2):
#    list1[i] = 'z'
#train_text = ''.join(list1)
#print("Train set =", train_text[0:100])

# Create objects for training, validation and testing batch generation.
train_batches = BatchGenerator(train_text, BATCH_SIZE, SEQ_LENGTH, vocabulary_size)

# Get first training batch.
batch = train_batches.next()
print(len(batch))
print(batch[0].shape)
#print("Batch = ", batch)
#print(batches2string(batch))
#print("batch len = num of enrollings",len(batch))
#for i in range(num_unrollings):
#    print("i = ", i, "letter=", batches2string(batch)[0][i][0], "bits = ", batch[i][0])


# For validation  - process the whole text as one big batch.
VALID_BATCH_SIZE = int(np.floor(valid_size/SEQ_LENGTH))
valid_batches = BatchGenerator(valid_text, VALID_BATCH_SIZE, SEQ_LENGTH, vocabulary_size)
valid_batch = valid_batches.next()
#print (VALID_BATCH_SIZE)
#print(len(valid_batch))
#print(valid_batch[0].shape)

# For texting  - process the whole text as one big batch.
TEST_BATCH_SIZE = int(np.floor(test_size/SEQ_LENGTH))
test_batches = BatchGenerator(test_text, TEST_BATCH_SIZE, SEQ_LENGTH, vocabulary_size)
# Get single batch! 
test_batch = test_batches.next()


2
(10, 59)


In [31]:
# Definition of the cell computation.
def controller_cell(input_, # input x
                    memory_input_, # read vector from the memory returned by previous cell
                    prev_output_, # output of the previous cell
                    prev_cell_state_, # previous cell state
                    prev_read_weights_batch_, # read weights from previous time state (t-1) 
                    prev_update_weights_batch_, # update weights from previous time state (t-1)
                    name_):
    """Create a controller with local memory cell"""
    """First dimensions of each of the computational nodes below is "derrived" from BATCH_SIZE"""
    with tf.name_scope(name_):

        with tf.name_scope("LSTM"):
            # LSTM cell equations according to Christopher Olah blog.
            # colah.github.io/posts/2015-08-Understanding-LSTMs/
            # Concatenate intyp x with h_prev ("prev output") TODO: and memory.
            i_h_m = tf.concat([input_, prev_output_], 1)

            # Calculate forget, input and output gate activations.
            forget_gate = tf.sigmoid(tf.matmul(i_h_m, Wf) + bf, name="Forget_gate")
            input_gate = tf.sigmoid(tf.matmul(i_h_m, Wi) + bi, name="Input_gate")
            output_gate = tf.sigmoid(tf.matmul(i_h_m, Wo) + bo, name="Output_gate")

            # Update of the cell state C~.
            cell_update = tf.tanh(tf.matmul(i_h_m, Wc) + bc, name="Cell_update")
            # New cell state C.
            cell_state = tf.add(forget_gate * prev_cell_state_, input_gate * cell_update, name = "Cell_state")
            # Calculate h - "output".
            cell_output = output_gate * tf.tanh(cell_state)

        # Read from the memory.
        with tf.name_scope("Read_head"):
            # Normalize batch.
            norm_batch = tf.nn.l2_normalize(cell_output,1)
            # Normalize memory.
            norm_memory = tf.nn.l2_normalize(memory,0)
            # Calculate cosine similarity.
            similarity_batch = tf.matmul(norm_batch, norm_memory)
            # Calculate read weights based on similarity.
            read_weights_batch = tf.nn.softmax(similarity_batch)

            # Add histograms to TensorBoard.
            norm_batch_hist = tf.summary.histogram("norm_batch", norm_batch)
            norm_memory_hist = tf.summary.histogram("norm_memory", norm_memory)
            similarity_batch_hist = tf.summary.histogram("cosine_similarity_batch", similarity_batch)
            read_weights_batch_hist = tf.summary.histogram("read_weights_batch", read_weights_batch)
        
        with tf.name_scope("Memory_output"):
            # Calcualte read vector.
            memory_output_batch = tf.tensordot(read_weights_batch, tf.transpose(memory), axes=1, name="Memory_output_batch_r")   
            # Add histograms to TensorBoard.
            memory_output_batch_hist = tf.summary.histogram("memory_output_batch", memory_output_batch)

        with tf.name_scope("Write_head"):
            # "Truncation scheme to update the least-used positions".
            # First, find (size-n) top elements (in each "batch sample"/head separatelly).
            top = tf.nn.top_k(-prev_update_weights_batch_, N_SMALLEST)
            # To get boolean True/False values, you can first get the k-th value and then use tf.greater_equal:
            kth = tf.reduce_min(top.values, axis=1, keep_dims=True)
            top2 = tf.greater_equal(-prev_update_weights_batch_, kth)
            # And finally - cast it to n smallest elements.
            prev_smallest_lru_weights = tf.cast(top2, tf.float32)

            #write_weights_seq_batch.append(prev_smallest_lru_weights)
            write_weights_batch = tf.add(tf.sigmoid(alpha) * prev_read_weights_batch_,
                                   (1.0 - tf.sigmoid(alpha)) * prev_smallest_lru_weights,
                                   name="Write_weights_ww")
                #test_batch.append(top)
            # Add histograms to TensorBoard.
            smallest_lru_weight_batch_hist = tf.summary.histogram("smallest_lru_weight_batch", prev_smallest_lru_weights)
            write_weights_batch_hist = tf.summary.histogram("write_weights_batch", write_weights_batch)

        with tf.name_scope("Update_head"):
            # This relies on prev. weights and will be used in fact in the NEXT step.
            update_weights_batch = tf.add(GAMMA * prev_update_weights_batch_,
                                            read_weights_batch + write_weights_batch,
                                            name="Update_weights_uw")
            # Add histograms to TensorBoard.
            update_weights_batch_hist = tf.summary.histogram("update_weights_batch", update_weights_batch)

            
    with tf.name_scope("Memory_update"):
        # Perform single update for each sequence/batch.
        memory_update_batch = tf.tensordot(tf.transpose(cell_output), write_weights_batch, axes=1)
        # Add dependendency control - first prediction?
        #with tf.control_dependencies([prediction_batch]):
        # Update the memory
        memory_update_op = memory.assign(memory + memory_update_batch)
        # Memory "truncation".
        memory_trunk_op = memory.assign(tf.tanh(memory))

        # Add histograms to TensorBoard.
        memory_update_batch_hist = tf.summary.histogram("memory_update_batch", memory_update_batch)
        memory_hist = tf.summary.histogram("memory_before_truncation", memory_update_op)
        memory_trunk_hist = tf.summary.histogram("memory_after_truncation", memory_trunk_op)
            
    return memory_output_batch, cell_output, cell_state

print("Cell definition OK")

Cell definition OK


###  Definition of tensor graph

In [32]:
# Reset graph - just in case.
tf.reset_default_graph()

# Memory.
memory = tf.Variable(tf.truncated_normal(shape=[HIDDEN_SIZE, MEMORY_SLOTS]), trainable=False, name="Memory_M")
# Latest vs LRU ratio.
alpha = tf.Variable(tf.truncated_normal(shape=[1]), name="Alpha")

# 0. Previous variables.
with tf.name_scope("Previous_variables"):
    # Create "read vectors" (in fact batch).
    read_vectors_seq_batch = list()    
    for i_seq in range(SEQ_LENGTH):
        read_vectors_seq_batch.append(tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="Read_vector_r"))    

    # Placeholders for previous weights.
    prev_read_weights_seq_batch = list()    
    prev_update_weights_seq_batch = list()    
    for i_seq in range(SEQ_LENGTH):
        prev_read_weights_seq_batch.append(tf.placeholder(tf.float32, shape=[None, MEMORY_SLOTS], name="Prev_rw"))
        prev_update_weights_seq_batch.append(tf.placeholder(tf.float32, shape=[None, MEMORY_SLOTS], name="Prev_uw"))

# 1. Placeholders for inputs.
with tf.name_scope("Input_data"):
    # Define input data buffers.
    data_buffers = list()
    for _ in range(SEQ_LENGTH + 1):
        # Collect placeholders for inputs/labels: Batch x Vocab size.
        data_buffers.append(tf.placeholder(tf.float32, shape=[None, vocabulary_size], name="data_buffers"))
    print ("data_buffers shape =", data_buffers[0].shape)

    # Sequence of batches.
    input_seq_batch = data_buffers[:SEQ_LENGTH]
    print ("Seq length  =", len(input_seq_batch))
    print ("Batch shape =", input_seq_batch[0].shape)

    # Labels are pointing to the same placeholders!
    # Labels are inputs shifted by one time step.
    labels_seq_batch = data_buffers[1:]  
    # Concatenate targets into 2D tensor.
    target_batch = tf.concat(labels_seq_batch, 0)

    # Add histograms to TensorBoard.
    input_seq_batch_hist = tf.summary.histogram("input_seq_batch", input_seq_batch)

# 2. Unrolled controller ops.
with tf.name_scope("Controller"):
    # Define parameters:
    # Input gate: input, previous output, and bias.
    Wf = tf.Variable(tf.truncated_normal([vocabulary_size+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wf")
    bf = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bf")

    # Forget gate: input, previous output, and bias.
    Wi = tf.Variable(tf.truncated_normal([vocabulary_size+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wi")
    bi = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bi")

    # Memory cell: input, state and bias.                             
    Wc = tf.Variable(tf.truncated_normal([vocabulary_size+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wc")
    bc = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bc")

    # Output gate: input, previous output, and bias.
    Wo = tf.Variable(tf.truncated_normal([vocabulary_size+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wo")
    bo = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bo")

    # Placeholders for "zero" (the oldest) state and output: Batch x Hidden size.
    init_controller_output = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_controller_output")
    init_controller_state = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_controller_state")
    # Placeholder for "zero" memory read: Batch X Hidden (TODO: memory?) size.
    init_memory_output = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_memory_read")

    # Unrolled LSTM.
    # Build outpus of size SEQ_LENGTH.
    controller_outputs_batch_seq = list()
    memory_outputs_batch_seq = list()
    # "Link" oldest statte and output to placeholders.
    controller_output = init_controller_output
    controller_state = init_controller_state
    memory_output = init_memory_output
    # For every buffer in input sequence batch buffers...
    for i in range(SEQ_LENGTH):
        # ... add cell...     
        memory_output, controller_output, controller_state = controller_cell(
            input_seq_batch[i], 
            memory_output, 
            controller_output, 
            controller_state, 
            prev_read_weights_seq_batch[i],
            prev_update_weights_seq_batch[i],
            "cell_"+str(i))
        # .. add controller buffer to outputs...
        controller_outputs_batch_seq.append(controller_output)
        memory_outputs_batch_seq.append(memory_output)
        # .. and set memory input of (i+1) cell to i-th read vector buffer.
        #memory_input = read_vectors_seq_batch[i]
        
    # Add histograms to TensorBoard.
    controller_outputs_batch_seq_hist = tf.summary.histogram("controller_outputs_batch_seq", controller_outputs_batch_seq)
    memory_outputs_batch_seq_hist = tf.summary.histogram("memory_outputs_batch_seq", memory_outputs_batch_seq)
    memory_hist = tf.summary.histogram("memory", memory)

# 3. Output ops.
with tf.name_scope("Output"):
    # Concatenate controller hidden state with the read vector.
    coutput_rvector_seq_batch = list()    
    for i_seq in range(SEQ_LENGTH):
        coutput_rvector_seq_batch.append(tf.concat([controller_outputs_batch_seq[i_seq], 
                                                 memory_outputs_batch_seq[i_seq]], 1, name="Concat_coutput_rvector"))    
    # Add histograms to TensorBoard.
    coutput_rvector_seq_batch_hist = tf.summary.histogram("coutput_rvector_seq_batch", coutput_rvector_seq_batch)

    output_batch = tf.concat(controller_outputs_batch_seq, 0) 
    #output_batch = tf.concat(controller_outputs_batch_seq, 0)
 
    # Output layer weights and biases.
    w = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, vocabulary_size], -0.1, 0.1), name="w")
    b = tf.Variable(tf.zeros([vocabulary_size]), name="b")

    # Logits.
    logits_batch = tf.nn.xw_plus_b(output_batch, w, b, name = "Final_FC")
    # Add fully connected softmax layer on top - predictions.
    prediction_batch = tf.nn.softmax(logits_batch)
    
# 4. Loss ops.
with tf.name_scope("Loss"):
    # Loss function(s) - one for every output generated by every LSTM cell.
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=target_batch, logits=logits_batch))
    # Add loss summary.
    loss_summary = tf.summary.scalar("loss", loss)

# 5. Training ops.  
with tf.name_scope("Optimization"):
    # Learning rate decay.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.1, global_step, 5000, 0.9, staircase=True)
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    # Gradient clipping.
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

# Merge all summaries.
merged_summaries = tf.summary.merge_all()

print("Graph definition OK")

data_buffers shape = (?, 59)
Seq length  = 1
Batch shape = (?, 59)
Graph definition OK


In [33]:
def create_feed_dict(set_type_):
    """Creates a dictionaries for different sets: maps data onto Tensor placeholders."""
    feed_dict = dict()
    
    #if set_type_=="train":
    # Get next batch and create a feed dict.
    next_batch = train_batches.next()
    # Feed batch to input buffers.
    for i in range(SEQ_LENGTH + 1):
        feed_dict[data_buffers[i]] = next_batch[i]

    # Set previous weights of read and write heades.
    for i in range(SEQ_LENGTH):
        feed_dict[prev_read_weights_seq_batch[i]] = prev_rw_seq_batch[i]
        feed_dict[prev_update_weights_seq_batch[i]] = prev_uw_seq_batch[i]

    # Reset "init" state and output of controller.
    feed_dict[init_controller_output] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
    feed_dict[init_controller_state] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
    feed_dict[init_memory_output] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
            
    #elif set_type_=="valid":
    #    for i in range(SEQ_LENGTH + 1):
    #        feed_dict[data_buffers[i]] = valid_batch[i]

    # TODO: HOW TO VALIDATE !! when update/write depends on the previous batch??
    
    #else: # test
    #    for i in range(SEQ_LENGTH + 1):
    #        feed_dict[data_buffers[i]] = test_batch[i]
        
       
    return feed_dict

print("Feed_dict definition OK")

Feed_dict definition OK


### Session execution

In [34]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

print("Log dir CLEARED")

Log dir CLEARED


In [36]:
# How often the test loss on validation batch will be computed. 
summary_frequency = 100

# Create session to execute graph.
sess=tf.InteractiveSession()

# Create summary writers, point them to LOG_DIR.
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize global variables.
tf.global_variables_initializer().run()
print('Variables initialized')


# Create initial previous read and update - full of zeros. 
prev_rw_seq_batch = list()
prev_uw_seq_batch = list()
for i in range(SEQ_LENGTH):
    prev_rw_seq_batch.append(np.zeros([BATCH_SIZE, MEMORY_SLOTS]))
    prev_uw_seq_batch.append(np.zeros([BATCH_SIZE, MEMORY_SLOTS]))

num_steps = train_size // (BATCH_SIZE*SEQ_LENGTH) #70001
print("Number of iterations per epoch =", num_steps)
for step in range(num_steps):
    input_seq_batch_, memory_, summaries, _, loss_, lr_  = sess.run([
        input_seq_batch, memory, merged_summaries, optimizer, loss, learning_rate],
        feed_dict=create_feed_dict("train"))#batch_seq))
    
    # Add summary.
    train_writer.add_summary(summaries, step*BATCH_SIZE*SEQ_LENGTH)
    train_writer.flush()

    # Every (100) steps collect statistics.
    if step % summary_frequency == 0:
        # Print loss from last batch.
        print('Training set BPC at step %d: %0.5f learning rate: %f' % (step, loss_, lr_))
    
        # Validation set BPC.
        #v_summaries, v_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("valid"))
        #print("Validation set BPC: %.5f" % v_loss)
        #valid_writer.add_summary(v_summaries, step*BATCH_SIZE*SEQ_LENGTH)
        #valid_writer.flush()
    # End of statistics collection

# Test set BPC.
#print("Calculating BPC on test dataset")
#t_summary, t_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("test"))
#print("Final test set BPC: %.5f" % t_loss)
#test_writer.add_summary(t_summary, step*BATCH_SIZE*SEQ_LENGTH)
#test_writer.flush()
    
# Close writers and session.
train_writer.close()
valid_writer.close()
test_writer.close()
sess.close() 

Variables initialized
Number of iterations per epoch = 510161
Training set BPC at step 0: 4.07831 learning rate: 0.100000
Training set BPC at step 100: 3.00187 learning rate: 0.100000
Training set BPC at step 200: 2.26890 learning rate: 0.100000
Training set BPC at step 300: 2.49882 learning rate: 0.100000
Training set BPC at step 400: 2.54054 learning rate: 0.100000
Training set BPC at step 500: 2.76042 learning rate: 0.100000
Training set BPC at step 600: 2.09428 learning rate: 0.100000
Training set BPC at step 700: 2.44707 learning rate: 0.100000
Training set BPC at step 800: 2.96052 learning rate: 0.100000
Training set BPC at step 900: 2.92975 learning rate: 0.100000
Training set BPC at step 1000: 3.27989 learning rate: 0.100000
Training set BPC at step 1100: 2.97388 learning rate: 0.100000
Training set BPC at step 1200: 2.43505 learning rate: 0.100000
Training set BPC at step 1300: 2.35510 learning rate: 0.100000
Training set BPC at step 1400: 2.15735 learning rate: 0.100000
Train

Training set BPC at step 13000: 1.71142 learning rate: 0.081000
Training set BPC at step 13100: 2.43050 learning rate: 0.081000
Training set BPC at step 13200: 2.07703 learning rate: 0.081000
Training set BPC at step 13300: 2.50570 learning rate: 0.081000
Training set BPC at step 13400: 1.96270 learning rate: 0.081000
Training set BPC at step 13500: 2.52034 learning rate: 0.081000
Training set BPC at step 13600: 2.62678 learning rate: 0.081000
Training set BPC at step 13700: 2.39482 learning rate: 0.081000
Training set BPC at step 13800: 2.45431 learning rate: 0.081000
Training set BPC at step 13900: 3.16822 learning rate: 0.081000
Training set BPC at step 14000: 2.98580 learning rate: 0.081000
Training set BPC at step 14100: 2.34203 learning rate: 0.081000
Training set BPC at step 14200: 2.16297 learning rate: 0.081000
Training set BPC at step 14300: 1.64667 learning rate: 0.081000
Training set BPC at step 14400: 2.08813 learning rate: 0.081000
Training set BPC at step 14500: 2.93596 

KeyboardInterrupt: 